# Extracción de datos desde APIs con Python


## Conceptos básicos de APIs REST y consumo de APIs en Python

### ¿Qué son las API?

- Son un conjuntos de reglas y protocolos que permiten a diferentes aplicaciones comunicarse entre sí. 
- Se utilizan para acceder a datos de servicios web, aplicaciones en la nube, redes sociales, entre otros. 
- Proporcionan una forma estructurada y segura de obtener datos de sistemas externos.

![API](../Images/API.png)

## Extracción de datos JSON desde APIs públicas y privadas

![API_Spotify](../Images/API_Spotify.png)

In [ ]:
import pandas as pd # manipular
import os # path
import json # manipular la respuesta de la api
import requests # para conectar a la api
import re # limpiar datos regex

# %%
path = os.getcwd()
path_credenciales = os.path.join(path, 'Credentials', 'credentials.json')
with open(path_credenciales) as file:
    credenciales = json.load(file)['Credentials']
    client_id = credenciales['client_id']
    client_secret = credenciales['client_secret']
# %%
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret':client_secret
}
token_url = 'https://accounts.spotify.com/api/token'
response = requests.post(token_url, data=data)
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Token de acceso:", access_token)
else:
    print("Error al solicitar el token de acceso:", response.text)


track = ''

# extract data from all tracks
def get_track_info(track):
    url = f'https://api.spotify.com/v1/audio-features/{track}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    response = requests.get(url, headers=headers)
    return response.json()



Consumo de la API de [mockaroo](https://www.mockaroo.com/), que es una herramienta que permite generar datos de prueba en diferentes formatos, como JSON, CSV, SQL y Excel.

In [2]:
import requests
headers = {'X-Api-Key': 'd67d54e0'}
response = requests.get('https://my.api.mockaroo.com/mkt_data_cf.json', headers=headers)

In [3]:
response.json()

[{'Fecha': '4/6/2020',
  'Nombre de la Campaña': 'Honorable',
  'Region': 'Australia',
  'Clicks': 6533,
  'Impresiones': 78913,
  'Views': 946856,
  'Costo': 99},
 {'Fecha': '2/3/2021',
  'Nombre de la Campaña': 'Rev',
  'Region': 'Africa',
  'Clicks': 5467,
  'Impresiones': 27498,
  'Views': 616990,
  'Costo': 256},
 {'Fecha': '9/21/2021',
  'Nombre de la Campaña': 'Mr',
  'Region': 'Africa',
  'Clicks': 6056,
  'Impresiones': 33352,
  'Views': 560650,
  'Costo': 162},
 {'Fecha': '6/15/2020',
  'Nombre de la Campaña': 'Honorable',
  'Region': 'Asia',
  'Clicks': 3720,
  'Impresiones': 95237,
  'Views': 426874,
  'Costo': 632},
 {'Fecha': '4/3/2022',
  'Nombre de la Campaña': 'Dr',
  'Region': 'South America',
  'Clicks': 4076,
  'Impresiones': 96455,
  'Views': 449235,
  'Costo': 865},
 {'Fecha': '8/7/2021',
  'Nombre de la Campaña': 'Honorable',
  'Region': 'Europe',
  'Clicks': 7967,
  'Impresiones': 72543,
  'Views': 979497,
  'Costo': 92},
 {'Fecha': '7/16/2021',
  'Nombre de la 

### Pandas JSON

In [4]:
import pandas as pd
from io import StringIO

In [5]:
pd.read_json(StringIO(response.text))

,Fecha,Nombre de la Campaña,Region,Clicks,Impresiones,Views,Costo
0,4/6/2020,Honorable,Australia,6533,78913,946856,99
1,2/3/2021,Rev,Africa,5467,27498,616990,256
2,9/21/2021,Mr,Africa,6056,33352,560650,162
3,6/15/2020,Honorable,Asia,3720,95237,426874,632
4,4/3/2022,Dr,South America,4076,96455,449235,865
5,8/7/2021,Honorable,Europe,7967,72543,979497,92
6,7/16/2021,Honorable,Africa,8228,80083,73313,978
7,5/4/2021,Rev,Africa,4313,83498,914504,865
8,1/27/2022,Ms,North America,3504,63773,952178,202
9,7/14/2022,Mrs,Asia,3170,10486,952208,42


In [6]:
import os
import json

path_data = os.path.join(os.getcwd(), 'data') 
ejemplo1_path = os.path.join(path_data, 'ejemplo1.json')
ejemplo2_path = os.path.join(path_data, 'ejemplo2.json')
ejemplo3_path = os.path.join(path_data, 'ejemplo3.json')

#### Ejemplo 1

In [7]:
pd.read_json(ejemplo1_path)

,nombre,productos
0,Tienda de Electrónica,"{'nombre_prod': 'Laptop', 'precio': 800, 'marc..."
1,Tienda de Electrónica,"{'nombre_prod': 'Teléfono', 'precio': 500, 'ma..."


#### json_normalize

**¿Qué hace `pd.json_normalize`?**

En esencia, esta función de Pandas toma datos JSON potencialmente complejos y anidados, y los transforma en un DataFrame plano y estructurado, que es mucho más fácil de manipular y analizar.

**Desglose de los parámetros:**

* `data`: Este es el JSON que contiene tus datos.

* `record_path='productos'`: Aquí le estás diciendo a Pandas: "Dentro de mi diccionario `data`, busca una lista llamada 'productos'. Cada elemento dentro de esa lista representa una fila en mi DataFrame final".

* `meta=['nombre']`:  Con esto indicas: "Además de los datos dentro de 'productos', quiero incluir una columna adicional llamada 'nombre' en mi DataFrame. El valor de esta columna será el valor asociado a la clave 'nombre' en el nivel superior de mi diccionario `data`".  En nuestro ejemplo, esto significa que cada fila tendrá el nombre de la tienda ("Tienda de Electrónica") junto con los detalles del producto.


In [8]:
data = json.load(open(ejemplo1_path))

pd.json_normalize(data, record_path='productos', meta=['nombre'])

,nombre_prod,precio,marca,nombre
0,Laptop,800,Marca A,Tienda de Electrónica
1,Teléfono,500,Marca B,Tienda de Electrónica


#### Ejemplo 2

In [9]:
pd.read_json(ejemplo2_path)

,usuario
nombre,Juan
edad,30
direcciones,"[{'calle': 'Calle Principal 123', 'ciudad': 'C..."
compras,"[{'producto': 'Libro', 'precio': 20, 'fecha': ..."


In [10]:
data2 = json.load(open(ejemplo2_path))
pd.json_normalize(
    data2
    )

,usuario.nombre,usuario.edad,usuario.direcciones,usuario.compras
0,Juan,30,"[{'calle': 'Calle Principal 123', 'ciudad': 'C...","[{'producto': 'Libro', 'precio': 20, 'fecha': ..."


In [11]:
df_direcciones = pd.json_normalize(
    data2['usuario'],
    record_path=['direcciones'],
    meta=['nombre', 'edad',]
    )
df_direcciones

,calle,ciudad,id,nombre,edad
0,Calle Principal 123,Ciudad A,1,Juan,30
1,Avenida Secundaria 456,Ciudad B,2,Juan,30


In [12]:
df_compras = pd.json_normalize(
    data2['usuario'],
    record_path=['compras'],
    meta=['nombre', 'edad',]
    )
df_compras

,producto,precio,fecha,id,nombre,edad
0,Libro,20,2020-01-01,1,Juan,30
1,Película,15,2020-01-02,2,Juan,30


#### Ejemplo 3

In [13]:
pd.read_json(ejemplo3_path)

,escuela
nombre,Escuela Primaria
grados,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [14]:
data3 = json.load(open(ejemplo3_path))
pd.json_normalize(
    data3['escuela'],)

,nombre,grados
0,Escuela Primaria,"[{'nombre': 'Primer Grado', 'alumnos': [{'nomb..."


In [15]:
pd.json_normalize(
    data3['escuela']['grados'],
        record_path='alumnos',
        meta=['nombre'],
        meta_prefix='grado_'
)

,nombre,edad,grado_nombre
0,Ana,6,Primer Grado
1,Pedro,7,Primer Grado
2,María,7,Segundo Grado
3,Luis,8,Segundo Grado
